<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#List-all-columns-in-Database" data-toc-modified-id="List-all-columns-in-Database-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>List all columns in Database</a></span></li><li><span><a href="#List-all-tables-with-a-specific-column" data-toc-modified-id="List-all-tables-with-a-specific-column-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>List all tables with a specific column</a></span></li><li><span><a href="#Summary-of-all-foreign-key-constraints" data-toc-modified-id="Summary-of-all-foreign-key-constraints-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Summary of all foreign key constraints</a></span></li><li><span><a href="#Query-on-the-system-catalog-views,-you-can-get-a-list-of-all-FK-relationships-and-how-the-link-two-tables-(and-what-columns-they-operate-on)" data-toc-modified-id="Query-on-the-system-catalog-views,-you-can-get-a-list-of-all-FK-relationships-and-how-the-link-two-tables-(and-what-columns-they-operate-on)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Query on the system catalog views, you can get a list of all FK relationships and how the link two tables (and what columns they operate on)</a></span></li><li><span><a href="#Data-Dictionary-Queries" data-toc-modified-id="Data-Dictionary-Queries-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Data Dictionary Queries</a></span></li></ul></div>

# List all columns in Database

In [ ]:
select schema_name(tab.schema_id) as schema_name,
    tab.name as table_name, 
    col.column_id,
    col.name as column_name, 
    t.name as data_type,    
    col.max_length,
    col.precision
from sys.tables as tab
    inner join sys.columns as col
        on tab.object_id = col.object_id
    left join sys.types as t
    on col.user_type_id = t.user_type_id
order by schema_name,
    table_name, 
    column_id;

# List all tables with a specific column

In [ ]:
SELECT c.name AS ColName, t.name AS TableName
FROM sys.columns c
    JOIN sys.tables t ON c.object_id = t.object_id
WHERE c.name LIKE '%COLUMN_NAME_HERE%';

# Summary of all foreign key constraints

In [ ]:
"SELECT
    c.CONSTRAINT_NAME,
    cu.TABLE_NAME AS ReferencingTable, cu.COLUMN_NAME AS ReferencingColumn,
    ku.TABLE_NAME AS ReferencedTable, ku.COLUMN_NAME AS ReferencedColumn
FROM INFORMATION_SCHEMA.REFERENTIAL_CONSTRAINTS c
INNER JOIN INFORMATION_SCHEMA.CONSTRAINT_COLUMN_USAGE cu
ON cu.CONSTRAINT_NAME = c.CONSTRAINT_NAME
INNER JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE ku
ON ku.CONSTRAINT_NAME = c.UNIQUE_CONSTRAINT_NAME"


In [ ]:
select schema_name(tab.schema_id) as [schema_name], 
    pk.[name] as pk_name,
    substring(column_names, 1, len(column_names)-1) as [columns],
    tab.[name] as table_name
from sys.tables tab
    inner join sys.indexes pk
        on tab.object_id = pk.object_id 
        and pk.is_primary_key = 1
   cross apply (select col.[name] + ', '
                    from sys.index_columns ic
                        inner join sys.columns col
                            on ic.object_id = col.object_id
                            and ic.column_id = col.column_id
                    where ic.object_id = tab.object_id
                        and ic.index_id = pk.index_id
                            order by col.column_id
                            for xml path ('') ) D (column_names)
order by schema_name(tab.schema_id),
    pk.[name]

# Query on the system catalog views, you can get a list of all FK relationships and how the link two tables (and what columns they operate on)

In [ ]:
SELECT
    fk.name 'FK Name',
    tp.name 'Parent table',
    cp.name, cp.column_id,
    tr.name 'Refrenced table',
    cr.name, cr.column_id
FROM 
    sys.foreign_keys fk
INNER JOIN 
    sys.tables tp ON fk.parent_object_id = tp.object_id
INNER JOIN 
    sys.tables tr ON fk.referenced_object_id = tr.object_id
INNER JOIN 
    sys.foreign_key_columns fkc ON fkc.constraint_object_id = fk.object_id
INNER JOIN 
    sys.columns cp ON fkc.parent_column_id = cp.column_id AND fkc.parent_object_id = cp.object_id
INNER JOIN 
    sys.columns cr ON fkc.referenced_column_id = cr.column_id AND fkc.referenced_object_id = cr.object_id
ORDER BY
    tp.name, cp.column_id

# Data Dictionary Queries

In [ ]:
https://dataedo.com/blog/useful-sql-server-data-dictionary-queries-every-dba-should-have